In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

bokuwakira_final_train_math_path = kagglehub.dataset_download('bokuwakira/final-train-math')
bokuwakira_classification_path = kagglehub.dataset_download('bokuwakira/classification')
bokuwakira_simple_qa_path = kagglehub.dataset_download('bokuwakira/simple-qa')
bokuwakira_reasoning_path = kagglehub.dataset_download('bokuwakira/reasoning')

print('Data source import complete.')


100%|██████████| 1.50M/1.50M [00:00<00:00, 33.2MB/s]

Extracting files...


100%|██████████| 48.1k/48.1k [00:00<00:00, 56.2MB/s]

Extracting files...


100%|██████████| 232k/232k [00:00<00:00, 77.7MB/s]

Extracting files...


100%|██████████| 203k/203k [00:00<00:00, 78.8MB/s]

Extracting files...
Data source import complete.


In [3]:
bokuwakira_final_train_math_path

'/root/.cache/kagglehub/datasets/bokuwakira/final-train-math/versions/1'

In [ ]:
from collections import Counter
import re
def extract_labels(text):
        matches = re.findall(r'\b(positive|negative|neutral)\b', text.lower())
        return list(set(matches))
def clean_label(text):
        match = re.search(r'\b(positive|negative|neutral)\b', text.lower())
        return match.group(1) if match else 'unknown'
def extract_answer(text):
        matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
        return matches[-1] if matches else None
def extract_target(text):
        for line in reversed(text.splitlines()):
            if line.strip().startswith("####"):
                return line.strip().replace("####", "").strip()
        return None
def split_cot_and_answer(self, text):
        lines = text.strip().splitlines()
        cot_lines = [line for line in lines if not line.strip().startswith("####")]
        answer = extract_target(text)
        return "\n".join(cot_lines), answer
def generate_output(prompt):
        response = model.chat(messages=[
            {"role": "user", "content": prompt}
        ])
        return response["choices"][0]["message"]["content"]
def self_consistency(prompt, num_samples=5):
        outputs = []
        for _ in range(num_samples):
            outputs.append(generate_output(prompt))
        return outputs
def majority_vote(outputs):
        counts = Counter(outputs)
        return counts.most_common(1)[0][0], counts
def cosine_similarity_score(pred, target):
        emb1 = comp_model.encode(pred, convert_to_tensor=True)
        emb2 = comp_model.encode(target, convert_to_tensor=True)
        return float(util.cos_sim(emb1, emb2)[0][0])
def find_top_k_tasks(text, k=3):
        q = vectorizer.transform([text])
        scores = (q @ X.T).flatten()
        top = scores.argsort()[::-1][:k]
        return [task_lib[i] for i in top]
def expand_thoughts(prompt, n=3):
        thoughts = []
        for _ in range(n):
            thoughts.append(generate_output(prompt))
        return thoughts

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

inputs = tokenizer("What is 12 * 9?", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is 12 * 9?
<|question_end|>Answer: 108



In [ ]:
model.to('cuda')

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2560,), eps=1

# Classification

In [ ]:
import json
with open('/kaggle/input/classification/classification_instruction_data_en.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

['reasoning (1).json']


In [ ]:
data[0]

{'instruction': "Classify the following text into 'positive', 'neutral', or 'negative'.",
 'input': 'between 1900 and 1920 where did most of the migrants to the united states come from',
 'output': 'neutral'}

In [ ]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]
    return None
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [ ]:
true_labels = []
for i in range(200):
    true_labels.append(data[i]['output'])

In [ ]:
from collections import Counter

print(Counter(true_labels))

Counter({'neutral': 157, 'positive': 38, 'negative': 5})


In [ ]:
import re
def clean_label(text):
    text = text.lower().strip()
    for label in ["positive", "neutral", "negative"]:
        if label in text:
            return label
    return "UNKNOWN"


Direct Zero-shot

In [ ]:
acc = 0
true_results = []
results = []

for i in range(200):
    zero_shot_prompt = f"""Sentence: {data[i]['input']}
Sentiment (neutral, positive, negative):"""


    # Tokenize prompt
    inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to("cuda")
    input_len = inputs["input_ids"].shape[1]  # Chiều dài prompt token

    # Generate output
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # Lấy phần output chỉ sau prompt
    generated_ids = outputs[0][input_len:]
    de = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    label = clean_label(de.lower())

    if label == data[i]['output']:
        acc += 1
        true_results.append(label)

    print("Generated:", de)
    print("Target:", data[i]['output'])
    print("Output:", label)

    results.append(label)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: neutral
Target: neutral
Output: neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: positive
Output: The sentiment of the sentence is positive.
Target: neutral
Output: positive


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Generated: neutral
Target: positive
Output: neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: neutral
Target: neutral
Output: neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: neutral
Target: neutral
Output: neutral
Generated: positive
Target: positive
Output: positive


In [ ]:
from collections import Counter
print(acc/200)
print(Counter(true_results))
print(Counter(results))

0.465
Counter({'neutral': 85, 'positive': 6, 'negative': 2})
Counter({'neutral': 119, 'positive': 48, 'negative': 21, 'UNKNOWN': 12})


Zero-shot CoT

In [ ]:
acc = 0
true_results = []
results = []
for i in range(200):
    zero_shot_prompt = f"""Sentence: "{data[i]['input']}"
Classify the sentiment of this sentence as one of: neutral, positive, or negative.
Return only the sentiment label without any explanation or additional text.
Sentiment:"""

    inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to("cuda")
    input_len = inputs["input_ids"].shape[1]
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    generated_ids = outputs[0][input_len:]
    de = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    label = clean_label(de.lower())

    if label == data[i]['output']:
        acc += 1
        true_results.append(label)

    print("Generated:", de)
    print("Target:", data[i]['output'])
    print("Output:", label)

    results.append(label)
print(acc/200)
print(Counter(true_results))
print(Counter(results))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: neutral
Target: neutral
Output: neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: Neutral
##OUTPUT
neutral
Target: neutral
Output: neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Generated: positive
A: Positive
Target: positive
Output: positive


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: negative
A: Negative
Target: neutral
Output: negative


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated: Positive
##OUTPUT
Positive
Target: neutral
Output: positive
Generated: positive
Target: positive
Output: positive
0.39
Counter({'neutral': 49, 'positive': 27, 'negative': 2})
Counter({'positive': 93, 'neutral': 60, 'negative': 46, 'UNKNOWN': 1})


In [ ]:
print(acc/200)
print(Counter(true_results))
print(Counter(results))

0.39
Counter({'neutral': 49, 'positive': 27, 'negative': 2})
Counter({'positive': 93, 'neutral': 60, 'negative': 46, 'UNKNOWN': 1})


In [ ]:
from collections import Counter
def self_consistency(prompt, num_samples=5, max_len=100):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        input_len = inputs["input_ids"].shape[1]
        output = model.generate(
            **inputs,
            max_new_tokens=max_len,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        generated_ids = output[0][input_len:]
        de = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        outputs.append(de)
    return outputs
def majority_vote(outputs):
    outputs = [clean_label(output.lower()) for output in outputs]
    counts = Counter(outputs)
    return counts.most_common(1)[0][0], counts

In [ ]:
def zero_shot_CoT(text):
    return f"""Sentence: "{text}"
Classify the sentiment of this sentence as one of: neutral, positive, or negative.
Return only the sentiment label without any explanation or additional text.
Sentiment:"""

In [ ]:
prompt = zero_shot_CoT(data[0]['input'])
print(prompt)
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentence: "between 1900 and 1920 where did most of the migrants to the united states come from"
Classify the sentiment of this sentence as one of: neutral, positive, or negative.
Return only the sentiment label without any explanation or additional text.
Sentiment:
neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


neutral
neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## INPUT
I am extremely satisfied with
negative
"""

# %%
# Let
Self-Consistent Answer: neutral
All votes: Counter({'neutral': 3, 'UNKNOWN': 1, 'negative': 1})


Zero-shot CoT + SC

In [ ]:
def zero_shot_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [ ]:
acc = 0
true_results = []
results = []
for i in range(100):
    best_answer, all_votes = zero_shot_CoT_SC(data[i], do_print=True)
    if clean_label(best_answer.lower()) == data[i]['output']:
        acc = acc + 1
        true_results.append(clean_label(best_answer.lower()))
    results.append(clean_label(best_answer.lower()))
print(acc/200)
print(Counter(true_results))
print(Counter(results))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Self-consistent answer:  UNKNOWN
All votes:  Counter({'UNKNOWN': 5})


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Self-consistent answer:  UNKNOWN
All votes:  Counter({'UNKNOWN': 5})


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Self-consistent answer:  UNKNOWN
All votes:  Counter({'UNKNOWN': 5})


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Self-consistent answer:  UNKNOWN
All votes:  Counter({'UNKNOWN': 5})


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [ ]:
print(acc/100)

0.96


In [ ]:
def select_best_path(thoughts, text, do_print=False):
    prompt = f"""
You are an expert in emotional reasoning. Analyze the emotional sentiment of the following sentence using the reasoning paths provided.

Context: "{text}"

Reasoning paths:
{chr(10).join([f"{i+1}. {t}" for i, t in enumerate(thoughts)])}

Instructions:
1. Carefully review the reasoning paths above. Focus on emotional polarity, tone, contrast, and subjectivity.
2. Select the most insightful and accurate reasoning path. Avoid shallow or purely literal interpretations.
3. Based on the best reasoning path, provide your answer in **this exact JSON format**:

{{
  "final_sentiment": "positive" | "negative" | "neutral",
  "reasoning": "Your explanation here based on the selected reasoning path."
}}

Important:
- Do NOT output only the sentiment.
- Your output must be a single valid JSON block as shown above.
"""


    if do_print:
        print('Prompt:\n', prompt)

    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(
        **input,
        max_new_tokens=350,
        do_sample=False
    )
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

In [ ]:
    def expand_thoughts( prompt, n=3, max_len=50):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        input_len = inputs["input_ids"].shape[1]
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        generated_ids = output[0][input_len:]
        thoughts = [tokenizer.decode(generated_ids, skip_special_tokens=True).strip() for output in outputs]
        return thoughts

In [ ]:
import re

def get_best_thought(thoughts, text, do_print=False):
    output = select_best_path(thoughts, text, do_print=do_print)
    if len(output) < 3:
        match = re.search(r"\b(\d+)\b", output)
        index = int(match.group(1)) - 1
        if 0 <= index < len(thoughts):
            return thoughts[index]
    else:
        return output


def expand_thoughts(prompt, n=3):
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = [model.generate(**inputs, max_length=150) for _ in range(n)]
    return [tokenizer.decode(o[0], skip_special_tokens=True) for o in outputs]

def recursive_expand_tree(prompt, depth, breadth, context, do_print=False):
    if depth == 0:
        return [prompt]

    base_prompt = f"""
        Use tree-of-thought reasoning to explore solutions.

        Question: {context}

        Partial Reasoning:
        {prompt}
        Now expand with next steps:
        """

    expanded = expand_thoughts(base_prompt, n=breadth)
    expanded = list(set(expanded))
    if do_print:
        print(f"[Depth {depth}] Base prompt:\n{base_prompt}")
        print(f"[Depth {depth}] Got thoughts:\n", expanded)

    tree = []
    for thought in expanded:
        sub_tree = recursive_expand_tree(thought, depth - 1, breadth, context, do_print)
        tree.extend(sub_tree)
    return tree


def select_best_path(thoughts, text, do_print=False):
    prompt = f"""Given a question and several reasoning paths, choose the most logical one.

    Question: {text}

    Candidates:
    {chr(10).join([f"{i + 1}. {t}" for i, t in enumerate(thoughts)])}

    Reply with the best option, justify briefly. Final answer is ?"""
    if do_print:
            print('prompt:\n', prompt)
    print(thoughts)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = model.generate(**inputs, max_length=300)
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

def zero_shot_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = zero_shot_ToT(text)
    if do_print:
            print('Root_prompt:\n', root_prompt)
    tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, do_print=do_print)
    best = select_best_path(tree, text, do_print)
    return best
def few_shots_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = few_shots_ToT(text)

    if do_print:
        print('[Root prompt]')
        print(root_prompt)

    tree = []
    for _ in range(breadth):
        single_tree = recursive_expand_tree(root_prompt, depth, breadth, text, do_print)
        tree.extend(single_tree)

    best_path = get_best_thought(tree, text, do_print)
    return best_path

In [ ]:
def recursive_expand_tree(prompt, depth, breadth, do_print=False):
    if depth == 0:
        return [prompt]

    expanded = expand_thoughts(prompt, n=breadth, max_len=100)
    if do_print:
        print(f"[Depth {depth}] Expand prompt:\n{prompt}")
        print(f"[Depth {depth}] Got thoughts:\n", expanded)

    tree = []
    for thought in expanded:
        sub_tree = recursive_expand_tree(thought, depth - 1, breadth, do_print)
        tree.extend(sub_tree)

    return tree

In [ ]:
def zero_shot_ToT(text):
    return f"""Classify the sentiment of the sentence below as one of: positive, neutral, or negative.

Sentence: "{text}"

Think step by step considering emotional tone, contrast, and subjectivity.

Final answer: (only the label)"""


Few-shots direct

In [ ]:
def few_shots_direct(text):
    return f"""Classify the sentiment of each sentence as positive, neutral, or negative.
Sentence: I love this! → positive
Sentence: This is terrible. → negative
Sentence: It works as expected. → neutral
Sentence: {text} →"""


In [ ]:
from collections import Counter

acc = 0
results = []
true_results = []

for i in range(100):
    prompt = few_shots_direct(data[i]['input'])
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    input_len = inputs['input_ids'].shape[1]

    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    generated_ids = outputs[0][input_len:]
    decoded = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    label = clean_label(decoded.lower())

    if label == data[i]['output']:
        acc += 1
        true_results.append(label)

    results.append(label)
    print(decoded)

print(f"Accuracy: {acc/100:.2f}")
print("All predictions:", Counter(results))
print("Correct predictions:", Counter(true_results))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


not applicable (no mention of migration in the sentence)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


neutral OUTPUT: The sentiment of the sentences are:

Positive
Negative
Ne


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


neutral OUTPUT: Positive
Negative
Neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


positive (assuming the tone is humorous and playful)


KeyboardInterrupt: 

Few-shots CoT

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
  few_shots_CoT =  f"""
    Classify the sentiment of each sentence as positive, neutral, or negative. Let's explain step by step.
    Sentence: I love this! → Reason: Strong positive words → Answer: positive
    Sentence: This is terrible. → Reason: Strong negative tone → Answer: negative
    Sentence: It works as expected. → Reason: Neutral tone and wording → Answer: neutral
    Sentence: {data[i]['input']} → Reason: Let's think step by step. The sentiment is """
  inputs = tokenizer(few_shots_CoT, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_new_tokens=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if clean_label(de.lower()) == data[i]['output']:
    acc = acc+1
    true_results.append(clean_label(de.lower()))
  results.append(clean_label(de.lower()))
print(acc/200)
counts = Counter(results)
print(counts)
counts = Counter(true_results)
print(counts)

In [ ]:
  def few_shots_CoT(text):
    return f"""
You're a sentiment classifier assistant. Classify the sentiment of the sentence into one of the following labels: **negative**, **positive**, or **neutral**. Use chain-of-thought reasoning to explain your answer clearly. Re-evaluate your classification using reasoning, as if double-checking with another LLM.

Examples:

Q: Although I had high hopes, this product was a huge disappointment.
Let's think step by step.
1. The person had high hopes.
2. But the product was a disappointment.
3. The overall sentiment is negative.
A: negative

Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?
Let's think step by step.
1. This is a factual and technical question.
2. It expresses no emotion.
3. Therefore, the sentiment is neutral.
A: neutral

Q: I hate this chat
Let's think step by step.
1. The word "hate" expresses strong negative emotion.
2. Therefore, the sentiment is negative.
A: negative

Q: Efficient and fast
Let's think step by step.
1. Both "efficient" and "fast" are positive qualities.
2. Therefore, the sentiment is positive.
A: positive

Q: {text}
Let's think step by step. A: The sentiment is (negative, positive or neutral)?
"""



In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
  inputs = tokenizer(few_shots_CoT(data[i]['input']), return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_new_tokens=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  results.append(clean_label(de.lower()))
  if clean_label(de.lower()) == data[i]['output']:
    acc = acc+1
    true_results.append(clean_label(de.lower()))
print('Accuracy ', acc/200)
counts = Counter(true_results)
print(counts)
counts = Counter(results)
print(counts)

Few-shots CoT + SC

In [ ]:
def few_shots_CoT_SC(text, output, num_samples=5, max_len=100, do_print=False):
    prompt = few_shots_CoT(text)
    samples = self_consistency(prompt, num_samples, max_len)

    total_cos = 0
    scores = []

    for sample in samples:
        score = cosine_similarity_score(sample, output)
        total_cos += score
        scores.append(score)
        if do_print:
            print('Sample:', sample)
            print('Cosine:', score)
            print()

    avg_score = total_cos / len(samples) if samples else 0
    return samples, avg_score


In [ ]:
def few_shots_CoT_SC(text, num_samples=5, max_len=50, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('Answers: ', samples)
            print('Self-consistent answer: ', best_answer)
            print('All votes: ', all_votes)
        return best_answer, all_votes

In [ ]:
acc = 0
results = []
true_results = []
for i in range(200):
    best_answer, all_votes = few_shots_CoT_SC(data[i], do_print=True)
    results.append(clean_label(best_answer.lower()))
    if clean_label(best_answer.lower()) == data[i]['output']:
        acc = acc + 1
        true_results.append(clean_label(best_answer.lower()))
print(acc/200)
print(Counter(true_results))
print(Counter(results))

In [ ]:
# def few_shots_ToT(text):
#         return f"""You are a sentiment analysis assistant. For each sentence, explain your reasoning and then classify it as **positive**, **negative**, or **neutral**.

#             Example 1:
#             Sentence: "The movie was boring but had a good ending."
#             Reasoning: Boring → negative, Good ending → positive. The ending left a stronger impression → Sentiment: positive

#             Example 2:
#             Sentence: "I expected more, but I guess it’s fine."
#             Reasoning: Disappointment shows unmet expectations, but “it’s fine” shows acceptance. Mixed feelings → Sentiment: neutral

#             Example 3:
#             Sentence: "The product is overpriced and doesn’t work well."
#             Reasoning: Overpriced and malfunctioning product indicates frustration → Sentiment: negative

#             Now analyze the following sentence:
#             Sentence: "{text}"
#             Reasoning:"""

In [ ]:
# acc = 0
# results = []
# true_results = []
# for i in range(200):
#     prompt = few_shots_ToT(data[i]['input'])
#     input = tokenizer(prompt, return_tensors='pt').to('cuda')
#     output = model.generate(
#         **input,
#         max_new_tokens=200,
#         num_beams=5,
#         no_repeat_ngram_size=2,
#         early_stopping=True
#     )
#     de = tokenizer.decode(output[0], skip_special_tokens=True)
#     if clean_label(de.lower()) == data[i]['output']:
#         acc = acc+1
#         true_results.append(clean_label(de.lower()))
#     results.append(clean_label(de.lower()))
# print('Accuracy', acc/200)
# print(Counter(true_results))
# print(Counter(results))

In [ ]:
# def few_shots_ToT_expanded( text, depth=2, breadth=3, do_print=False):
#       root_prompt = f"""You are a sentiment analysis assistant. For each sentence, explain your reasoning and then classify it as **positive**, **negative**, or **neutral**.

#             Example 1:
#             Sentence: "The movie was boring but had a good ending."
#             Reasoning: Boring → negative, Good ending → positive. The ending left a stronger impression → Sentiment: positive

#             Example 2:
#             Sentence: "I expected more, but I guess it’s fine."
#             Reasoning: Disappointment shows unmet expectations, but “it’s fine” shows acceptance. Mixed feelings → Sentiment: neutral

#             Example 3:
#             Sentence: "The product is overpriced and doesn’t work well."
#             Reasoning: Overpriced and malfunctioning product indicates frustration → Sentiment: negative

#             Now analyze the following sentence:
#             Sentence: "{text}"
#             Reasoning:"""
#         tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, do_print=do_print)
#         best_path = select_best_path(tree, text)
#         return best_path

In [ ]:
# acc = 0
# results = []
# true_results = []
# for i in range(200):
#     result = few_shots_ToT_expanded(data[i]['input'], breadth = 3)
#     if clean_label(result.lower()) == data[i]['output']:
#         acc = acc+1
#         true_results.append(clean_label(result.lower()))
#     results.append(clean_label(result))
# print(acc/200)
# print(Counter(true_results))
# print(Counter(results))

In [ ]:
# acc = 0
# results = []
# true_results = []
# for i in range(200):
#     result = few_shots_ToT_expanded(data[i]['input'], depth = 2, breadth = 2)
#     if result.lower() == data[i]['output']:
#         acc = acc+1
#         true_results.append(result.lower())
#     results.append(result)
# print(acc/200)
# print(Counter(true_results))
# print(Counter(results))

**Zero-shot**

Direct zero-shot

In [ ]:
print(generate_output(zero_shot_direct(data[0]['input'])))

 The sentiment of the sentence is neutral.


In [ ]:
results = []
true_results = []
for i in range(100):
    answer = generate_output(zero_shot_direct(data[i]['input']))
    label = clean_label(answer)
    if label == data[i]['output']:
        true_results.append(label)
    results.append(label)
print('True results: ', Counter(true_results))
print('Results: ', Counter(results))

True results:  Counter({'neutral': 77, 'positive': 4, 'negative': 1})
Results:  Counter({'neutral': 90, 'positive': 6, 'negative': 2, 'unknown': 2})


Zero-shot CoT

In [ ]:
run(data[0]['input'],'Zero-shot CoT')

' The sentence is a question that asks for information about the migration patterns of the United States between 1900 and 1920. The subject of the sentence is "migrants," the verb is "came," and the object is "to the United States." The question is asking where the majority of these migrants came from during that time period.\n\nThe sentiment of the sentence is neutral. It is a factual question that seeks information without expressing an opinion or emotion.'

In [ ]:
results = []
true_results = []
for i in range(100):
    answer = generate_output(zero_shot_CoT(data[i]['input']))
    label = clean_label(answer)
    if label == data[i]['output']:
        true_results.append(label)
    results.append(label)
print('True results: ', Counter(true_results))
print('Results: ', Counter(results))

KeyboardInterrupt: 

Zero-shot CoT + SC

In [ ]:
run(data[0]['input'], type = 'Zero-shot CoT + Self-consistency', num_samples = 5, max_len = 200)

In [ ]:
results = []
true_results = []
for i in range(100):
    answer = run(data[i]['input'], type = 'Zero-shot CoT + Self-consistency', num_samples = 5, max_len = 200)
    label = clean_label(answer)
    if label == data[i]['output']:
        true_results.append(label)
    results.append(label)
print('True results: ', Counter(true_results))
print('Results: ', Counter(results))

In [ ]:
results = []
true_results = []
for i in range(100):
    answer = run(data[i]['input'], type = 'Zero-shot CoT + Self-consistency', num_samples = 3, max_len = 200)
    label = clean_label(answer)
    if label == data[i]['output']:
        true_results.append(label)
    results.append(label)
print('True results: ', Counter(true_results))
print('Results: ', Counter(results))

Zero-shot ToT (basic)

In [ ]:
run(data[0]['input'], type = 'Zero-shot ToT', do_print=True)

In [ ]:
results = []
true_results = []
for i in range(100):
    answer = run(data[i]['input'], type = 'Zero-shot ToT', do_print=True)
    label = clean_label(answer)
    if label == data[i]['output']:
        true_results.append(label)
    results.append(label)
print('True results: ', Counter(true_results))
print('Results: ', Counter(results))

In [ ]:
# def few_shots_ToT(text):
#         return f"""You are a sentiment analysis assistant. For each sentence, explain your reasoning and then classify it as **positive**, **negative**, or **neutral**.

#             Example 1:
#             Sentence: "The movie was boring but had a good ending."
#             Reasoning: Boring → negative, Good ending → positive. The ending left a stronger impression → Sentiment: positive

#             Example 2:
#             Sentence: "I expected more, but I guess it’s fine."
#             Reasoning: Disappointment shows unmet expectations, but “it’s fine” shows acceptance. Mixed feelings → Sentiment: neutral

#             Example 3:
#             Sentence: "The product is overpriced and doesn’t work well."
#             Reasoning: Overpriced and malfunctioning product indicates frustration → Sentiment: negative

#             Now analyze the following sentence:
#             Sentence: "{text}"
#             Reasoning:"""

# Computation

In [5]:
import json
math = []
with open('/root/.cache/kagglehub/datasets/bokuwakira/final-train-math/versions/1/final_train_math.json', 'r', encoding='utf-8') as f:
    math.append(json.load(f))

In [6]:
math = math[0]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2560,), eps=1

In [ ]:
prompt = f"Q: {math[0]['input'] } A: Let's think step by step."
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_length=160)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? A: Let's think step by step. Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48/2 = 24 clips. To find the total number of clips sold in April and May, we add the number of clips sold in each month: 48 + 24 = 72 clips.



In [8]:
import re
import re

def extract_answer(text):
  if text !=None:
    for line in text.strip().splitlines()[::-1]:
        line_lower = line.lower()
        if re.search(r'\b(remaining|final|total|answer|so|therefore|thus)\b', line_lower):
            matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', line)
            if matches:
                num = matches[-1]
                if float(num) > 1:
                    return num

    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        num = matches[-1]
        if float(num) > 1:
            return num
    return None
  return None


def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

Zero-shot CoT

In [ ]:
acc = 0
results = []
for i in range(50):
  zero_shot_prompt = f"""You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.

Problem: {math[i]['input']}

Let's think step by step.

Answer:"""


  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_new_tokens=512,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )

  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print('Explanation: ', answer)
  print('Answer: ',extract_answer(answer))
  target = extract_target(math[i]['output'])
  print('Target: ', target)
  if extract_answer(answer) == target:
    acc = acc+1
  print('----------------------------------------------------------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Explanation:  You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.

Problem: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Let's think step by step.

Answer: 

Step 1: Identify the number of clips sold in April.
Natalia sold 48 clips in April to her friends.
Step 2: Determine how many clips she sold in May, which is half of what she sold the previous month.
In May, she sold 48/2 = 24 clips. 
Step 3: Calculate the total clips sold by adding the clips from both months.
Total clips sold = 48 (April) + 24 (May) = 72 clips.

Answer:  72
Target:  72
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Explanation:  You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.

Problem: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

Let's think step by step.

Answer: 
Step 1: Convert 50 minutes to hours. Since there are 60 minutes in an hour, 50 minutes is 50/60 = 0.83 hours.
Step 2: Multiply Weng's hourly rate by the number of hours she babysat. $12/hour * 0.833 hours = $10.
So, Weng earned $10 yesterday.

Answer:  10
Target:  10
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Explanation:  You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.

Problem: Carolyn practices the piano for 20 minutes a day and the violin for three times as long. If she practice six days a week, how many minutes does she spend practicing in a month with four weeks?

Let's think step by step.

Answer: 

Step 1: Calculate the time Carolyn spends practicing the violin.
She practices the violin three times longer than the piano, so she practices for 20 * 3 = 60 minutes each day.
Step 2: Multiply the daily practice time by the number of days in a week.
60 minutes/day * 6 days/week = 360 minutes/week
Step 3: Find the total practice time for a month by multiplying the weekly time by four weeks.
360 minutes * 4 weeks/month = 1440 minutes/month

So, Carolyn spends 1440 minutes, or 24 hours, practicing the piano and violin each month.

Answer:  60
Target:  1920
--------------------------------------------------

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Explanation:  You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.

Problem: The file, 90 megabytes in size, downloads at the rate of 5 megabytes per second for its first 60 megabytes, and then 10 megabytes per second thereafter. How long, in seconds, does it take to download entirely?

Let's think step by step.

Answer:

Step 1: Calculate the time it takes to download the first part of the file.
Time = Size / Rate
Time for first 60MB = 60MB / 5MB/s = 12 seconds

So, it takes 12 seconds for the first 60 MB to download.
 
Step 2: Now, let's calculate the time for the remaining 30MB.
Rate = 10MB per second
Time to download 30MB = 30MB / 10MB/second = 3 seconds 

Therefore, it will take a total of 12 seconds (first 60MB) + 3 seconds (remaining 30MB) = 15 seconds in total for the file to download completely.

Answer:  15
Target:  15
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Explanation:  You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.

Problem: Sam memorized six more digits of pi than Carlos memorized. Mina memorized six times as many digits of pi as Carlos memorized. If Mina memorized 24 digits of pi, how many digits did Sam memorize?

Let's think step by step.

Answer:

Step 1: Let's use a variable to represent the number of digits that Carlos remembered. We can call it C.
Step 2: According to the problem, Sam remembered C + 6 digits, and Mina remembered 6C digits. We are given that Mina remembers 24 digits, so we can write an equation: 6C = 24
Step 3: To find the value of C, we need to isolate it on one side of the equation. To do that, we can divide both sides by 6: C = 24 / 6
Step 4: Simplify the right side of

Answer:  24
Target:  10
----------------------------------------------------------
Explanation:  You are a helpful and smart math assistant.

Solve the foll

số lượng câu đúng:

18/50 = 0.36

18/50 =

In [ ]:
print(acc/50)

0.28


In [ ]:
from collections import Counter
import torch

def self_consistency(prompt, num_samples=5, max_len=150):
    outputs = []

    # Encode prompt 1 lần
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    input_len = inputs["input_ids"].shape[1]

    for _ in range(num_samples):
        output = model.generate(
            **inputs,
            max_new_tokens=max_len,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )
        new_tokens = output[0][input_len:]
        decoded = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
        outputs.append(decoded)

    return outputs


def majority_vote(outputs):
    for output in outputs:
        print("Output:\n", output)
        print()

    answers = [extract_answer(output) for output in outputs]
    answers = [a for a in answers if a is not None]  # bỏ các kết quả lỗi

    if not answers:
        return None, Counter()

    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts


Zero-shot CoT + SC

In [ ]:
def zero_shot_CoT(text):
        return f"""You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.

Problem: {math[i]['input']}

Let's think step by step.

Answer:"""
def zero_shot_CoT_SC(text, num_samples=5, max_len=200, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('All vote: ', all_votes)
            print('Best answer: ', best_answer)
        return best_answer, all_votes

In [ ]:
from collections import Counter
zero_shot_CoT_SC(math[0]['input'], num_samples = 5, do_print=True)

Output:
 Step 1: Betty needs $100 to buy the wallet.
Step 2: She has $50 (half of the money needed).
Step 3: Her parents give her an additional $15.
Step 4: Her grandparents give her twice as much as her parents, which is 2 * $15 = $30.
Step 5: Therefore, Betty has a total of $50 + $15 + $30 = $95.
Step 6: The difference between what she needs and what she has is $100 - $95 = $5.
Step 7: So, Betty still needs $5 to buy the wallet.

Output:
 Step 1: Find out how much money Betty currently has. 
Betty has half the money she needs. So, she has $100 / 2 = $50.

Step 2: Calculate the total money Betty received from her parents and grandparents. 
Betty's parents gave her $15 and her grandparents gave her twice that amount, so they gave her $15 * 2 = $30.

Step 3: Add the money Betty has and the money she received from her parents and grandparents. 
Total money Betty has = $50 + $30 = $80.

Step 4: Subtract the total money Betty has from the cost of the wallet. 
Money Betty still needs = $100

('95', Counter({'95': 1, '80': 1, '5': 1, '55': 1, '30': 1}))

In [ ]:
acc = 0
ap_acc = 0

for i in range(50):
    best_answer, all_votes = zero_shot_CoT_SC(math[i]['input'], do_print=True)
    target = extract_target(math[i]['output'])

    print('Target:', target)
    print('----------------------------------------------')

    if best_answer == target:
        acc += 1

    if target in all_votes:
        ap_acc += 1


Output:
 Step 1: Natalia sold 48 clips in April.
Step 2: In May, she sold half as many clips as she sold in April, which means she sold 48 / 2 = 24 clips.
Step 3: To find the total number of clips Natalia sold in April and May, we add the number of clips she sold in April and the number of clips she sold in May: 48 + 24 = 72 clips.

So, Natalia sold a total of 72 clips in April and May.

Output:
 Natalia sold 48 clips in April. In May, she sold half as many, which is 48 / 2 = 24 clips. Altogether, she sold 48 + 24 = 72 clips.

Output:
 Step 1: Natalia sold 48 clips in April.

Step 2: In May, she sold half as many clips, which is 48 / 2 = 24 clips.

Step 3: To find the total number of clips sold, we add the number of clips sold in April and May. So, 48 + 24 = 72 clips were sold altogether.

Therefore, Natalia sold 72 clips altogether in April and May.

Output:
 1. Natalia sold clips to 48 friends in April.
2. She sold half as many clips in May, which is 48 / 2 = 24 clips.
3. So, Natalia

In [ ]:
print(acc/50)

0.64


In [ ]:
print(ap_acc/50)

0.8


In [15]:
def generate_output(type, input, max_len=400):
        if type=='generation':
            output = model.generate(
                **input,
                max_length = max_len,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                num_return_sequences=1
            )
        else:
            output = model.generate(
                **input,
                max_length = max_len,
                max_new_tokens = max_len,
                num_beams=5,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
        return output

In [ ]:
# def zero_shot_ToT(text):
#     return f"""You are a helpful and accurate math assistant.

# Solve the following problem by considering at least three different solution paths. Show your reasoning and formulas clearly in each path.

# After analyzing the approaches, explain which one is the most reliable and why. Then give your final answer.

# Question: {text}
# Now, compare the approaches. Which one is the most accurate and why?

# Final Answer:
# """

# acc = 0

# for i in range(50):
#     prompt = zero_shot_ToT(math[i]['input'])
#     inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
#     input_len = inputs["input_ids"].shape[1]

#     output = model.generate(
#         **inputs,
#         max_new_tokens=500,
#         do_sample=False,
#         num_beams=3,                      # giữ ở mức hợp lý
#         early_stopping=True,
#         pad_token_id=tokenizer.eos_token_id
#     )

#     new_tokens = output[0][input_len:]
#     answer = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

#     predicted = extract_answer(answer)
#     target = extract_target(math[i]['output'])

#     print(f"Output #{i+1}:\n{answer}")
#     print('Predicted:', predicted)
#     print('Target:', target)
#     print('------------------------------------------------')

#     if predicted == target:
#         acc += 1

# print(f'ToT Accuracy: {acc}/50 = {acc/50:.2%}')


In [ ]:
print(acc/20)

Few-shots CoT

In [ ]:
acc  = 0
results = []
for i in range(3,23):
    few_shots_prompt = f"""
You are a helpful and accurate math assistant. Whenever you see a math expression, evaluate it using the function CALC(). Always write all intermediate steps explicitly.

Here are some examples:

Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A: Natalia sold CALC(48/2) = 24 clips in May.
Total clips = CALC(48 + 24) = 72.
#### 72

Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A: Weng earns CALC(12 / 60) = 0.2 dollars per minute.
In 50 minutes, she earned CALC(0.2 * 50) = 10 dollars.
#### 10

Q: {math[i]['input']}
A: Let's solve this step by step. Remember to use CALC() to evaluate all math.
"""


    inputs = tokenizer(few_shots_prompt, return_tensors='pt').to('cuda')
    input_len = inputs["input_ids"].shape[1]
    output = generate_output(input = inputs, type=None)
    new_tokens = output[0][input_len:]
    answer= tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    print('Explanation: ', answer)
    print('Answer: ',extract_answer(answer))
    target = extract_target(math[i]['output'])
    print('Target: ', target)
    if extract_answer(answer) == target:
      acc = acc+1
    print('----------------------------------------------------------')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  Today, Julie read 2 * 12 = <<2*12=24>>24 pages. 
The total number of pages she has read so far is 120 - 12 - 24 = 84 pages left. To read 50% of these pages (which is half), she needs to calculate 50/100 * 84 = 40.8. Since she can't read a fraction of a page, we round up to the nearest whole number, which is 41. So Julie should read 41 more pages to reach her goal of reading half the book tomorrow.
Answer:  40.8
Target:  42
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  James writes 3 * 2 * 52 = 312 pages in one year.
Answer:  312
Target:  624
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Explanation:  First, let's find out how long it takes Tim to bike to and from work each day. The round trip is 2 * 20 = 40 miles, so it will take him 40 / 25 = 1.6 hours, or 1 hour and 36 minutes. Since he does this for 5 days, he spends a total of 5 * (1 + 36/60) hours = 5.5 hours on his work commute each week. 
Next, we need to add in the time Tim spends biking on the weekend. He bikes for 200/25 = 8 hours. So, the total biking time for the week is 5 + 8 = 13 hours or 13.33 hours when rounded to two decimal places.
Answer:  13
Target:  16
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  Bella bought = 11 (snowflakes) + (11 + 9) (trucks)
= 11 + 20 = 31
She also bought (20 - 13) roses = 7
So, the total number of stamps she bought is 31 + 7 = 38.
Answer:  38
Target:  38
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  First, calculate the total number of snakes eaten by all the snakes: 6 * 5 = 30 snakes. 
Then, find out how much birds these snakes eat in total: 30 * 3 = 90 birds.  
Finally, figure out the amount of beetles these birds eat: 90 * 12 = 1080 beetles.
Answer:  90
Target:  1080
----------------------------------------------------------
Explanation:  First, we need to find out how many more letters Jamie has in her name than Samantha. We can do this by subtracting the number of letters in Jamie's first name (Jamie) from the total number (8).
Jamie - Jamie = 8 - 4 = 4
Now, let's use this information to set up an equation for the second part of the problem. 
Bobbie - 2 = 2 * Jamie
Substituting the value we found for Jamie (4) into the equation gives us
(Samantha's name length) - (2 letters taken off) + 4 letters = 16 letters
Simplifying this equation by adding 2 to both sides and combining like terms we get
CALC((16 + 2)) = 18
Therefore, Samantha has 18 - 6 = 12 letters left to

In [ ]:
print(7/20)

0.35


In [ ]:
def few_shots_CoT(text):
    return f"""Solve these math problems step by step:

Q1: A store offers a 30% discount on a $200 item. What's the price after discount?
Step 1: Calculate 30% of 200 = 0.3 * 200 = 60
Step 2: Subtract from 200 → 200 - 60 = 140
A1: $140

Q2: If a car travels 60 miles in 1.5 hours, what is its average speed in miles per hour?
Step 1: Use the formula: speed = distance / time
Step 2: speed = 60 / 1.5 = 40
A2: 40 miles per hour

Q3: John has 3 boxes. Each box contains 12 apples. He gives away 10 apples. How many apples does he have left?
Step 1: Total apples = 3 * 12 = 36
Step 2: Apples left = 36 - 10 = 26
A3: 26 apples

Q4: A shirt originally costs $80. It is marked up by 25%. What's the new price?
Step 1: Calculate markup = 25% of 80 = 0.25 * 80 = 20
Step 2: New price = 80 + 20 = 100
A4: $100

Now solve:
{text}
 Let's solve this step by step. Remember to evaluate all math.
"""

In [ ]:
acc = 0
for i in range(50):
    inputs = tokenizer(few_shots_CoT(math[i]['input']), return_tensors='pt').to('cuda')
    input_len = inputs["input_ids"].shape[1]
    output = generate_output(input = inputs, type=None)
    new_tokens = output[0][input_len:]
    answer= tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    print('Explanation: ', answer)
    print('Answer: ',extract_answer(answer))
    target = extract_target(math[i]['output'])
    print('Target: ', target)
    if extract_answer(answer) == target:
      acc = acc+1
    print('----------------------------------------------------------')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  April: 48 clips
 May: 1/2 * 48 = 24 clips 
 Total clips sold = 48 + 24 = 72 clips
Answer:  72
Target:  72
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  First, convert minutes to hours: 50/60 = 5/6 hours
Then, calculate her earnings: 12 * (5 / 6) = 10 dollars
So, Weng earned $10 yesterday.
Answer:  10
Target:  10
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Explanation:  First, find out how long Carolyn practices violin each day: 20 (minutes on piano) * 3 (times longer on violin) = <<20*3=60>>60 minutes
Then, calculate the total minutes she practices each week: (20 + 60) minutes/day * 6 days/week = 360 minutes per week
Finally, multiply the weekly minutes by the number of weeks in the month to get the monthly total: 360 * 4 weeks/month = 1,440 minutes
Answer:  360
Target:  1920
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  First, let's calculate the time it takes for the first part of the download. 
Time = Size / Rate
= 60 Megabytes / 5 MB/s = 12 seconds
For the second part, the file size is 90-60=30 Megabyte. The download rate is 10MB/second. So,
time = size / rate = 30MB / 10 MB /s= 3 seconds.


Answer: It takes 12 + 3 = 15 seconds to fully download the 90MB file.
Answer:  15
Target:  15
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  Let's denote the number of digits Carlos remembered as C. Then, Sam remembered C + 6 digits. We know from the problem that Sam also remembered 6 * C digits, which is 24. So, we can set up the following equation:

 6C = 24

 Solving for C, divide both sides of the equation by 6: 

 C = 4

 Now that we know the value of C (Carlos' memorization), substitute it back into the expressions for Sam's memorizations to find the answer.

 Sam remembers C+6 = 6+4 = 10 digits
Answer:  10
Target:  10
----------------------------------------------------------
Explanation:  First, let's find out the number of shells Ben collected. We know that Ben's collection is one-third of Laurie's. So,
Ben's shells = 1/3 of 36 = 12 shells
Next, we need to find Alan's shell collection, which is 4 times that of Ben’s. 
Alan's Shells = 4*12 = 48 shells
Answer:  48
Target:  48
----------------------------------------------------------


In [ ]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
task_samples = math[:100]
new_tasks = []
for task in task_samples:
    cot, answer = split_cot_and_answer(task["output"])
    task["chain_of_thought"] = cot
    task["answer"] = answer
    new_tasks.append(task)
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)

In [ ]:
def find_top_k_tasks(text, k=3):
        q = vectorizer.transform([text])
        scores = cosine_similarity(q, X).flatten()
        top = scores.argsort()[::-1][:k]
        return [new_tasks[i] for i in top]

In [ ]:
def find_similar_task(query):
    q = vectorizer.transform([query])
    idx = cosine_similarity(q, X).argmax()
    return new_tasks[idx]

In [ ]:
test_tasks = math[1000:]

In [ ]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}

                    """
    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt

Few-shots CoT + ART

In [ ]:
def few_shots_CoT_ART(query, k = 3):
    examples = find_top_k_tasks(query, k)
    prompt = build_prompt(examples, query)
    inputs = tokenizer(few_shots_CoT(math[i]['input']), return_tensors='pt').to('cuda')
    input_len = inputs["input_ids"].shape[1]
    output = generate_output(type=None,input = inputs)
    new_tokens = output[0][input_len:]
    answer= tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    return answer

In [ ]:
acc = 0
for i in range(50):
    answer = few_shots_CoT_ART(test_tasks[i]['input'])
    print('Explanation: ', answer)
    print('Answer: ',extract_answer(answer))
    target = extract_target(math[i]['output'])
    print('Target: ', target)
    if extract_answer(answer) == target:
      acc = acc+1
    print('----------------------------------------------------------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  April: 48 clips
 May: 1/2 * 48 = 24 clips 
 Total clips sold = 48 + 24 = 72 clips
Answer:  72
Target:  72
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  First, convert minutes to hours: 50/60 = 5/6 hours
Then, calculate her earnings: 12 * (5 / 6) = 10 dollars
So, Weng earned $10 yesterday.
Answer:  10
Target:  10
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Explanation:  First, find out how long Carolyn practices violin each day: 20 (minutes on piano) * 3 (times longer on violin) = <<20*3=60>>60 minutes
Then, calculate the total minutes she practices each week: (20 + 60) minutes/day * 6 days/week = 360 minutes per week
Finally, multiply the weekly minutes by the number of weeks in the month to get the monthly total: 360 * 4 weeks/month = 1,440 minutes
Answer:  360
Target:  1920
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  First, let's calculate the time it takes for the first part of the download. 
Time = Size / Rate
= 60 Megabytes / 5 MB/s = 12 seconds
For the second part, the file size is 90-60=30 Megabyte. The download rate is 10MB/second. So,
time = size / rate = 30MB / 10 MB /s= 3 seconds.


Answer: It takes 12 + 3 = 15 seconds to fully download the 90MB file.
Answer:  15
Target:  15
----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explanation:  Let's denote the number of digits Carlos remembered as C. Then, Sam remembered C + 6 digits. We know from the problem that Sam also remembered 6 * C digits, which is 24. So, we can set up the following equation:

 6C = 24

 Solving for C, divide both sides of the equation by 6: 

 C = 4

 Now that we know the value of C (Carlos' memorization), substitute it back into the expressions for Sam's memorizations to find the answer.

 Sam remembers C+6 = 6+4 = 10 digits
Answer:  10
Target:  10
----------------------------------------------------------
Explanation:  First, let's find out the number of shells Ben collected. We know that Ben's collection is one-third of Laurie's. So,
Ben's shells = 1/3 of 36 = 12 shells
Next, we need to find Alan's shell collection, which is 4 times that of Ben’s. 
Alan's Shells = 4*12 = 48 shells
Answer:  48
Target:  48
----------------------------------------------------------


In [ ]:
print(acc/50)

0.36


In [ ]:
def few_shots_CoT(text):
        return f"""You are a helpful and smart math assistant.

Solve the following problem step-by-step using formulas, and clearly show your reasoning.
Q1: A store offers a 30% discount on a $200 item. What's the price after discount?
Step 1: Calculate 30% of 200 = 0.3 * 200 = 60
Step 2: Subtract from 200 → 200 - 60 = 140
A1: $140

Q2: If a car travels 60 miles in 1.5 hours, what is its average speed in miles per hour?
Step 1: Use the formula: speed = distance / time
Step 2: speed = 60 / 1.5 = 40
A2: 40 miles per hour

Problem: {math[i]['input']}

Let's think step by step.

Answer:"""

In [ ]:
def few_shots_CoT_SC(text, num_samples=5, max_len=1000, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples)
        best_answer, all_votes = majority_vote(samples)
        if do_print:
            print('All vote: ', all_votes)
            print('Best answer: ', best_answer)
        return best_answer, all_votes

In [ ]:
acc = 0
ap_acc = 0
for i in range(50):
    best_answer, all_votes = few_shots_CoT_SC(text = math[i]['input'], do_print=True)
    target = extract_target(math[i]['output'])
    print('Target:', target)
    print('----------------------------------------------')
    if best_answer == target:
        acc += 1
    if target in all_votes:
        ap_acc += 1


Output:
 Step 1: Find out how many clips Natalia sold in May. Half of 48 is 24, so she sold 24 clips in May.
Step 2: Add the clips sold in April and May. 48 + 24 = 72
A3: Natalia sold 72 clips altogether in April and May.

Q3: Natalia needs to buy a new phone that costs $800. She has saved $400. How much more money does she need to save?

Let's think step by step.

Answer: 
Step 1: Subtract the amount saved from the cost of the phone. 800 - 400 = 400
A3: Natalia needs to save $400 more.

Q4

Output:
 Step 1: Natalia sold 48 clips in April.
Step 2: She sold half as many in May, which is 48 / 2 = 24 clips.
Step 3: To find the total, we add the number of clips from April and May. 48 + 24 = 72 clips.

So, Natalia sold 72 clips altogether in April and May.

Keep practicing, and soon, you'll be solving math problems as easily as a pro! Remember, every step in math is like a step in a dance - every move matters!

Output:
 Step 1: Natalia sold 48 clips in April.
Step 2: She sold half as many c

In [ ]:
print(acc/50)
print(ap_acc/50)

0.44
0.72


In [ ]:
def expand_thoughts(prompt, n=3, max_len=500):
        input = tokenizer(prompt, return_tensors='pt').to('cuda')
        outputs = model.generate(
            **input,
            max_length = max_len,
            num_return_sequences=n,
            do_sample = True,
            top_p=0.95,
            temperature=0.7
        )
        thoughts = [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
        return thoughts

In [ ]:
import re

def get_best_thought(thoughts, text, do_print=False):
    output = select_best_path(thoughts, text, do_print=do_print)
    if len(output) < 3:
        match = re.search(r"\b(\d+)\b", output)
        if match:
          index = int(match.group(1)) - 1
          if 0 <= index < len(thoughts):
              return thoughts[index]
          return None
    else:
        return output

In [ ]:
def zero_shot_ToT(text):
    return f"""You are a math assistant. For the question below, brainstorm multiple reasoning paths before concluding the answer.

Q: {text}
Thought 1:"""
def few_shots_ToT(text):
        return f"""
        You are a math assistant.\n
         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips in April.
        Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
        Thought 3: Total clips = CALC(48+24)=72.

        Q: {text}
        A:
        Let's think in steps, exploring each possibility:
        Thought 1:
        Thought 2:
        Thought 3:
        """

def expand_thoughts(prompt, n=3):
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    input_len = inputs["input_ids"].shape[1]

    outputs = [model.generate(
        **inputs,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id
    ) for _ in range(n)]

    return [
        tokenizer.decode(output[0][input_len:], skip_special_tokens=True).strip()
        for output in outputs
    ]
def recursive_expand_tree(prompt, depth, breadth, context, do_print=False, node_counter=[0], max_nodes=30):
    if depth == 0 or node_counter[0] >= max_nodes:
        return [prompt]

    base_prompt = f"""You are a math assistant. Use tree-of-thought reasoning to explore solutions.

Question: {context}

Partial Reasoning:
{prompt}
Now expand with next steps:"""

    expanded = expand_thoughts(base_prompt, n=breadth)
    expanded = list(set(expanded))

    if do_print:
        print(f"[Depth {depth}] Base prompt:\n{base_prompt}")
        print(f"[Depth {depth}] Got thoughts:\n", expanded)

    tree = []
    for thought in expanded:
        if node_counter[0] >= max_nodes:
            break
        node_counter[0] += 1
        sub_tree = recursive_expand_tree(thought, depth - 1, breadth, context, do_print, node_counter, max_nodes)
        tree.extend(sub_tree)
    return tree


def select_best_path(thoughts, text, do_print=False):
    prompt = f"""You are a math expert. Given a question and several reasoning paths, choose the most logical one.

Question: {text}

Candidates:
{chr(10).join([f"{i + 1}. {t}" for i, t in enumerate(thoughts)])}

Reply with only one best option, justify briefly. Final answer is ? (DO NOT offer followup exercises)"""

    if do_print:
        print('prompt:\n', prompt)

    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    input_len = inputs["input_ids"].shape[1]

    output = model.generate(
        **inputs,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0][input_len:], skip_special_tokens=True).strip()

def zero_shot_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = zero_shot_ToT(text)
    if do_print:
            print('Root_prompt:\n', root_prompt)
    tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, context=text, do_print=do_print, max_nodes=30)

    best_path = get_best_thought(tree, text, do_print)
    return best_path
def few_shots_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = few_shots_ToT(text)

    if do_print:
        print('[Root prompt]')
        print(root_prompt)

    tree = []
    for _ in range(breadth):
        single_tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, context=text, do_print=do_print, max_nodes=30)

        tree.extend(single_tree)

    best_path = get_best_thought(tree, text, do_print)
    return best_path


In [ ]:
    context = math[0]['input']
    answer = few_shots_ToT_expanded(math[0]['input'], depth = 2, breadth =2, do_print=True)
    target = extract_target(math[0]['output'])
    print('Output: ', answer)
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Root prompt]

        You are a math assistant.

         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips in April.
        Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
        Thought 3: Total clips = CALC(48+24)=72.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Let's think in steps, exploring each possibility:
        Thought 1:
        Thought 2:
        Thought 3:
        


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Depth 2] Base prompt:
You are a math assistant. Use tree-of-thought reasoning to explore solutions.

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Partial Reasoning:

        You are a math assistant.

         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips in April.
        Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
        Thought 3: Total clips = CALC(48+24)=72.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Depth 1] Base prompt:
You are a math assistant. Use tree-of-thought reasoning to explore solutions.

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Partial Reasoning:
Thought 1: Natalia sold 48 clips in April.
Thought 2: She sold half as many in May → 48/2=24 clips.
Thought 3: Total clips = 48+24=72.

Answer: Natalia sold 72 clips altogether in April and May.
Now expand with next steps:
[Depth 1] Got thoughts:
 ['Question: If Natalia sold the same number of clips in June as she did in May, how many clips did she sell in June?\n\nPartial Reasoning:\nThought 1: Natalia sold 24 clips in May.\nThought 2: She sold the same number in June → 24 clips.\n\nAnswer: Natalia sold 24 clips in June.\n\nQuestion: If Natalia sold the same number of clips in July as she did in June, how many clips did she sell in July?\n\nPartial Reasoning:\nThought 1: Natalia sold 24 clips in June.\

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Depth 2] Base prompt:
You are a math assistant. Use tree-of-thought reasoning to explore solutions.

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Partial Reasoning:

        You are a math assistant.

         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips in April.
        Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
        Thought 3: Total clips = CALC(48+24)=72.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Depth 1] Base prompt:
You are a math assistant. Use tree-of-thought reasoning to explore solutions.

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Partial Reasoning:
Thought 1: Natalia sold 48 clips in April.
Thought 2: She sold half as many in May → 48/2=24 clips.
Thought 3: Total clips = 48+24=72.

Answer: Natalia sold 72 clips altogether in April and May.
Now expand with next steps:
[Depth 1] Got thoughts:
 ['Question: If Natalia sold the same number of clips in June as she did in May, how many clips did she sell in June?\n\nPartial Reasoning:\nThought 1: Natalia sold 24 clips in May.\nThought 2: She sold the same number in June → 24 clips.\n\nAnswer: Natalia sold 24 clips in June.\n\nQuestion: If Natalia sold the same number of clips in July as she did in June, how many clips did she sell in July?\n\nPartial Reasoning:\nThought 1: Natalia sold 24 clips in June.\

Few-shots ToT expanded _--------------------------

In [ ]:
model.to('cuda')

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2560,), eps=1

In [ ]:
for i in range(3, 53):
    answer = few_shots_ToT_expanded(math[i]['input'], depth = 2, breadth =2, do_print=False)
    target = extract_target(math[i]['output'])
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

Output:  Solution:

1. 
        You are a math assistant.

         Use a tree-of-thought approach to break down complex problems by exploring different solution paths, then converge on the correct answer.

        Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
        A:
        Thought 1: Natalia sold 48 clips
Answer:  None
Target:  42
-------------------------------------------------------------------------------------
Output:  Solution:

The most logical reasoning path is Thought 3.

Explanation:

Thought 1: Natalia sold 48 clips in April.
Thought 2: She sold half as many in May → CALC(48/2)=24 clips.
Thought 3: Total clips = CALC(48+24)=72.

Thought 1 and 2 are incorrect because they do not consider the fact that Natalia sold half as many clips in
Answer:  72
Target:  624
-------------------------------------------------------------------------------------
Output:  Solut

In [ ]:
for i in range(3, 53):
    answer = zero_shot_ToT_expanded(math[i]['input'], depth = 2, breadth =2, do_print=False)
    target = extract_target(math[i]['output'])
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

Output:  Answer: Julie should read 36 pages tomorrow.

Explanation:

1. Julie read 12 pages yesterday, so she has 120 - 12 = 108 pages left to read.
2. Today, she read twice as many pages as yesterday, which is 2 * 12 = 24 pages.
3. Therefore, she has 108 - 24 = 84 pages left to read.
4. If she wants to read half of the remaining pages tomorrow, she should read 84 /
Answer:  84
Target:  42
-------------------------------------------------------------------------------------
Output:  A: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?

Answer: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?

Answer: James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?

Answer: James writes a 3-page letter to 2
Answer:  None
Target:  624
-------------------------------------------------------------------------------------
Output:  A:

Let's star

In [ ]:
for i in range(3, 53):
    answer = few_shots_ToT_expanded(math[i]['input'], depth = 1, breadth =3, do_print=False)
    target = extract_target(math[i]['output'])
    print('Output: ', answer)
    print('Answer: ', extract_answer(answer))
    print('Target: ', target)
    print('-------------------------------------------------------------------------------------')

Output:  Solution:

Thought 1:
Julie read 12 pages yesterday and 24 pages today. She has read 36 pages in total.
Half of the remaining pages is (120-36)/2 = 54 pages.
Therefore, Julie should read 54 pages tomorrow.

Follow-up exercises:
1. What if Julie wants to read one-third of the remaining pages tomorrow? How many pages should she read?
2. What if Julie read 15 pages yesterday instead
Answer:  54
Target:  42
-------------------------------------------------------------------------------------
Output:  Solution:

The most logical reasoning path is Thought 1.

Explanation:

Thought 1 is the most logical reasoning path because it is the most direct and efficient way to solve the problem. Natalia sold 48 clips in April and half as many in May, which is 24 clips. Therefore, the total number of clips sold in April and May is 48 + 24 = 72 clips.

Follow-up exercises:

1. What is the total
Answer:  72
Target:  624
----------------------------------------------------------------------------

#REASONING

In [ ]:
import os

base_path = "/root/.cache/kagglehub/datasets/bokuwakira/reasoning/versions/1"
print(os.listdir(base_path))  # In ra danh sách các file trong thư mục


['reasoning (1).json']


In [13]:
import json
data= []
file_path = bokuwakira_reasoning_path
with open('/root/.cache/kagglehub/datasets/bokuwakira/reasoning/versions/1/reasoning (1).json', "r", encoding="utf-8") as f:
    data = json.load(f)

In [14]:
from sentence_transformers import SentenceTransformer, util
comp_model = SentenceTransformer('all-MiniLM-L6-V2')
def cosine_similarity_score(pred, target):
    emb1 = comp_model.encode(pred, convert_to_tensor = True)
    emb2 = comp_model.encode(target, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2)[0][0])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Zero-shot direct

In [ ]:
def zero_shot_direct(text):
    return f"""Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.

{text}

Answer:"""


In [ ]:
data[0]

{'source': 'Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl ester of salicylic acid, originally isolated from the bark of willow trees, and a wide range of opiate painkillers like heroin are obtained by chemical modification of morphine obtained from the opium poppy. Popular stimulants come from plants, such as caffeine from coffee, tea 

In [ ]:
prompt = zero_shot_direct(data[0]['input'])
input = tokenizer(prompt, return_tensors='pt').to('cuda')
answer = tokenizer.decode(generate_output(type=None, input = input, max_len=1000)[0], skip_special_tokens=True).strip()
print(answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=1000) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.

Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl ester of salicylic acid, originally isolated from the bark of willow trees, and a wide range of opiate painkillers like heroin are obtained by chemical modif

In [ ]:
data[1]

{'source': "In this task, you are provided with an article of the legal acts. Your task is to classify it into three categories (Regulation, Decision and Directive) based on its content: 1) Regulation is a binding legislative act that must be applied in its entirety on a set date across all the member states (European Union countries). 2) Decision is binding on those to whom it is addressed (e.g. an European Union country or an individual company) and is directly applicable. 3) Directive is a legislative act that sets out a goal that all  must achieve. However, it is up to the individual countries to devise their own laws on how to reach these goals.\n\nProof that the special export tax mentioned in Articles 2 and 3 of Regulation (EEC) No 1234/71 has been paid shall be furnished to the competent authority of the importing Member State by presentation of movement certificate A.TR.1. In that case, one of the following entries shall be made in the 'Remarks' section by the competent author

In [ ]:
prompt = zero_shot_direct(data[1]['input'])
input = tokenizer(prompt, return_tensors='pt').to('cuda')
answer = tokenizer.decode(generate_output(type=None, input = input, max_len=500)[0], skip_special_tokens=True).strip()
print(answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.

In this task, you are provided with an article of the legal acts. Your task is to classify it into three categories (Regulation, Decision and Directive) based on its content: 1) Regulation is a binding legislative act that must be applied in its entirety on a set date across all the member states (European Union countries). 2) Decision is binding on those to whom it is addressed (e.g. an European Union country or an individual company) and is directly applicable. 3) Directive is a legislative act that sets out a goal that all  must achieve. However, it is up to the individual countries to devise their own laws on how to reach these goals.

Proof that the special export tax mentioned in Articles 2 and 3 of Regulation (EEC) No 1234/71 has been paid shall be furnished to the competent authority of the importing Member State by presentation of movement certi

In [ ]:
import re

def extract_answer(text, answer):
    pattern = re.escape(answer)
    match = re.search(pattern, text)
    if match:
        return True
    else:
        return False

def extract_target(text):
    match = re.search(r'Answer:\s*(.*)', text)
    if match:
        return match.group(1).strip()
    else:
        return None


In [ ]:
cosine = 0
for i in range(50):
  prompt = zero_shot_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  input_len = input['input_ids'].shape[1]
  output = generate_output(type=None, input = input)
  generate_ids = output[0][input_len:]
  answer = tokenizer.decode(generate_ids, skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print('CS: ', c)
  print('Answer: ', answer)
  print('Target: ', data[i]['output'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.6852311491966248
Answer:  Plants. The article states that many medicinal, recreational, stimulant and painkiller drugs are derived directly or indirectly from plant sources. Therefore, plants are the origin of some drugs and substances that affect the human body and mind. This is an example of common sense reasoning, which is based on general knowledge and observation of the natural world. It does not require any specific domain or factual information, but rather a logical and intuitive understanding of cause and effect.
Target:  Answer: from plants


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.5477254390716553
Answer:  Regulation
Explanation: The article mentions a specific tax that needs to be paid for a certain type of export, and it specifies that this tax is required by a particular regulation. This indicates that it falls under the category of a regulation, as regulations are binding laws that apply to all members of an organization or group. The other two categories, decision and directive, do not mention any specific taxes or regulations, but rather focus on setting goals and allowing for flexibility in achieving them. Therefore, the correct answer is Regulation.
Target:  Answer: Regulation


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

CS:  0.6713754534721375
Answer:  The article does not provide enough information to determine the source of the water. It only describes the process and history of well construction, but not the location or origin of groundwater. A possible explanation is that groundwater comes from rain or snow that seeps into the soil and rocks and fills the spaces between them, creating an underground reservoir of water that can be accessed by wells. However, this is not a definitive answer, as different regions may have different geological features and hydrological cycles that affect the availability and quality of underground water sources. Therefore, more information is needed to answer this question.
Target:  Answer: groundwater in underground aquifers


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.4141680598258972
Answer:  The correct answer is "none". This is because William believes that he has moved all of his belongings to a different location, so he thinks that there is nothing left for Mia to search for. However, this is not necessarily true, as Mia might still have some of William's belongings in her possession, or she might have moved them to another location herself. Therefore, we cannot assume that William knows the exact whereabouts of Mia's search, and we have to consider the possibility that she is looking for something else entirely. This question tests the reader's ability to infer the mental state of another person based on their actions and beliefs, which is a key aspect of theory of mind.
Target:  Answer: red_bottle


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.5869941711425781
Answer:  The correct option is A. 1.5
To solve this problem, we need to use the distributive property of multiplication over addition and subtraction. This means that we can multiply each term inside the parentheses by the factor outside and then add or subtract the results. For example, a*(b+c) = ab + ac. So we have:
7644=4648d-4557d
We can simplify this by combining the like terms, which are the terms that have the same variable and exponent. In this case, both terms have d as the variable, and they have an exponent of 1, so they are like. We can add them together by adding their coefficients. The coefficient of a term is the number in front of it that tells us how many of that term there are. A coefficient can be positive, negative, or zero, but it cannot be zero if the term has a variable with a nonzero exponent, because that would mean that there is no term with that variable at all. Adding the coefficients of d gives us the coefficient
Target:  Answer: 84


In [ ]:
# ACC:
print(cosine/50)

0.5695955181121826


Zero-shot CoT

In [ ]:
def zero_shot_CoT(text):
    return f"""Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.
  Q: {text}. A: Answer:... Let's think step by step"
Answer:"""

In [ ]:
acc=0
cosine = 0
for i in range(50):
  prompt = zero_shot_CoT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  input_len = input['input_ids'].shape[1]
  output = generate_output(type=None, input = input)
  generate_ids = output[0][input_len:]
  answer = tokenizer.decode(generate_ids, skip_special_tokens=True).strip()
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print('CS: ', c)
  print('Answer: ', answer)
  print('Target: ', data[i]['output'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.7964053153991699
Answer:  Plants. The article states that many medicinal, recreational and pain-relieving drugs are derived directly or indirectly from plant sources. Therefore, plants are the correct answer.
Target:  Answer: from plants


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.5170534253120422
Answer:  Regulation
Explanation: The article talks about a specific tax that needs to be paid when exporting goods, and it mentions a regulation that outlines the details of this tax. Therefore, this article falls under the category of regulation.
Target:  Answer: Regulation


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

CS:  0.7750667333602905
Answer:  The article states that water wells are excavations or structures that access underground groundwater. It also mentions that wells have been constructed for thousands of years and have varying levels of complexity. However, it does not mention where the water actually comes from. Therefore, we cannot determine the answer based on this information alone. We would need more information about the specific well in question to determine its water source.
Target:  Answer: groundwater in underground aquifers


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.43214085698127747
Answer:  1. First, we need to determine where William and Mia are in relation to each other at the beginning and end of their actions. At the start, they are both in different rooms (bathroom and living room) and William is holding a red bottle and a bucket, while Mia is not mentioned to be holding anything. However, by the end, Mia has entered William's room and they have both exited their respective rooms. This means that William has moved from his original location to Mia's location, and vice versa. Therefore, their current locations are the same as their starting locations, which are different from the original question's scenario of them being in separate rooms with different objects. Thus, the correct answer is (D) None of these, as there is no mention of Mia searching for anything in this scenario.
Target:  Answer: red_bottle


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.6724951863288879
Answer:  The correct answer is A. To solve this problem, we need to isolate d on one side of the equation. We can do this by subtracting 4657 from both sides, and then dividing by the coefficient of d, which is 4647. This gives us d = (7644 -
Target:  Answer: 84
CS:  0.5427908897399902
Answer:  The creature that is referred to as an "eft" is a bat. This can be inferred from the fact that bats are the only mammals capable of sustained flight, making them unique and distinct from other animals. Additionally, the use of the word "alternative" suggests that there may be other names for this creature, but the most commonly used one is "bat."
Target:  Answer: Eft


In [ ]:
print(cosine/50)
#ACC:

0.5368865970149637


Zero-shot CoT SC

In [ ]:
cosine = 0
def zero_shot_CoT_SC(text, output, num_samples=5, max_len=50, do_print=False):
        prompt = zero_shot_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        c = 0
        if do_print:
          for sample in samples:
            print('Sample : ', sample)
            c = c+ cosine_similarity_score(sample, output)
            print('Cs: ', cosine_similarity_score(sample, output))
            print()
        return samples, c

In [ ]:
cosine=0
for i in range(50):
  prompt, c = zero_shot_CoT_SC(text = data[i]['input'],output =  data[i]['explanation'], num_samples=3, max_len = 300, do_print=True)
  cosine = cosine + c
  print('Target: ', data[i]['output'])
  print('Explanation (target): ', data[i]['explanation'])
  print('--------------------------------------------------------------------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample :  Some medicines and recreational drugs come from plants. This is because the article mentioned that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. The article also mentioned that some drugs are simple derivatives of botanical natural products. Therefore, the correct answer is A.
Cs:  0.9271024465560913

Sample :  Some medicines and recreational drugs come from plants.

Explanation: The article mentions that some medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. It also mentions that other drugs are simple derivatives of botanical natural products.

2. Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.
  Q: Article: Phytochemistry is a branch of plant biochemistry primarily concerned

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample :  A: Answer:
Cs:  0.07388919591903687

Sample :  The regulation is binding in its entirety and directly applicable in all member states. This means that it must be applied in its entirety on a set date across all the member states.
Cs:  0.7010577321052551

Sample :  A: Answer: Let's think step by step
Cs:  0.05785932391881943

Target:  Answer: Regulation
Explanation (target):  This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
--------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Sample :  William thinks that Mia searches for the celery in the red_bucket.

Explanation: In the scenario where William's belief matches reality, the celery is in the red_bucket. Therefore, William believes that Mia would search for the celery in the red_bucket.
Cs:  0.7587578296661377

Sample :  A: William thinks that Mia searches for the celery in the red_bucket.
Cs:  0.666905403137207

Sample :  William thinks that Mia searches for the celery in the red_bucket.
Cs:  0.6800533533096313

Target:  Answer: red_bottle
Explanation (target):  Mia's belief about the location of the celery does not match reality, but William's belief (red_bottle) about Mia's belief matches reality.
--------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample :  Possible rewrite:

Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.

Question: Solve this math problem

7644 = 4648d - 4557d

Where d is a positive integer. Choose the correct answer.

A. Answer: 9
B. Answer: 10
C. Answer: 11
D. Answer: 12

Solution: The correct answer is B. 10.

To solve this problem, we can use the following steps:

- Subtract 7644 from both sides of the equation to get -1220 = -955d.
- Divide both sides by -955 to get d = 10.
Cs:  0.5955740213394165

Sample :  Possible rewrite:

Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.

Q: Solve this math problem

Solve 7644 = 4648*d - 4557*d for d. Choices:. A: Answer:... Let's think step by step.

To find the value of d, we need to isolate it on one side of the equation. We can do this by using the distributive property of multiplication over subtr

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample :  The creature that an eft is an alternative name for is a caterpillar.
Cs:  0.6700323820114136

Sample :  An eft is an alternative name for a worm. This can be deduced from the information provided in the paragraph, which states that an eft is the larval stage of a worm, and the word "eft" is used in the context of describing a larva.

Exercise: Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.
  Q: What is the main difference between an eft and a larva? Choices:. A: Answer:... Let's think step by step"
Answer: The main difference between an eft and a larva is that an eft specifically refers to the larval stage of a worm, while a larva can refer to the larval stage of any type of insect. This can be inferred from the sentence "An eft is the larval stage of a worm" in the paragraph.

Exercise: Read the article and answer the question based only on the information provided. Then explain your reaso

In [ ]:
print(cosine/150)

0.6067790711422761


Few-shots CoT SC

In [ ]:
cosine = 0
def few_shots_CoT_SC(text, output, num_samples=3, max_len=500, do_print=False):
        prompt = few_shots_CoT(text)
        samples = self_consistency(prompt, num_samples, max_len)
        c = 0
        if do_print:
          for sample in samples:
            print('Sample : ', sample)
            c = c+ cosine_similarity_score(sample, output)
            print('Cs: ', cosine_similarity_score(sample, output))
            print()
        return samples, c

In [ ]:
cosine=0
for i in range(2, 52):
  prompt, c = few_shots_CoT_SC(text = data[i]['input'],output =  data[i]['explanation'], num_samples=3, max_len = 500, do_print=True)
  cosine = cosine + c
  print('Target: ', data[i]['output'])
  print('Explanation (target): ', data[i]['explanation'])
  print('--------------------------------------------------------------------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample :  Lee surrendered to Grant to end the civil war.
Answer: Because metals conduct heat away from the skin faster

---

Q: What's the answer to that question: where did lee surrender to grant to end the civil war? Choices: 
A:
Let's think step by step.
Answer: Lee surrendered to Grant to end the civil war.
Answer: Because metals conduct heat away from the skin faster

---

Q: What's the answer to that question: where did lee surrender to grant to end the civil war? Choices: 
A:
Let's think step by step.
Answer: Lee surrendered to Grant to end the civil war.
Answer: Because metals conduct heat away from the skin faster

---

Q: What's the answer to that question: where did lee surrender to grant to end the civil war? Choices: 
A:
Let's think step by step.
Answer: Lee surrendered to Grant to end the civil war.
Answer: Because metals conduct heat away from the skin faster

---

Q: What's the answer to that question: where did lee surrender to grant to end the civil war? Choices: 
A:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [ ]:
def few_shots_direct(text):
    return f"""
Here are some examples:
Q: {data[0]['input']}
A: Answer: from plants. Explanation {data[0]['explanation']}
Q: {data[1]['input']}
A: {data[1]['output']}. Explanation {data[0]['explanation']}
Q: {text}
Read the article and answer the question based only on the information provided. Then explain your reasoning in 1-2 short sentences.
A: Answer:"""



In [60]:
def clear_gpu():
    import gc, torch
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()


In [61]:
clear_gpu()

Few-shots direct

In [ ]:
acc=0
cosine = 0
for i in range(2, 52):
  prompt = few_shots_direct(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  input_len = input['input_ids'].shape[1]
  output = generate_output(type=None, input = input)
  generate_ids = output[0][input_len:]
  answer = tokenizer.decode(generate_ids, skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print('CS: ', c)
  print('Answer: ', answer)
  print('Target: ', data[i]['output'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.7280873656272888
Answer:  Lee surrendered to Grant at Appomattox Court House, Virginia, on April 9, 1865, ending the American Civil War. Lee's surrender was the result of a series of events that led to his defeat at the Battle of Gettysburg in July 1863. After this defeat, Lee realized that he could not win the war and decided to surrender. Grant, who had been leading the Union army, accepted the surrender and Lee was taken as a prisoner of war. He was eventually released in 1868 and went on to live the rest of his life in relative obscurity. The surrender of Lee and his army was a significant event in American history and helped to bring an end to a devastating conflict that had lasted for more than four years.
Target:  Answer: Battle of Appomattox Court House


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.1737174093723297
Answer:  6 weeks
Explanation: The question asks us to find the difference between the two numbers. We can do this by subtracting the smaller number (2 weeks) from larger number(6 weeks). The answer is 4 weeks.
Target:  Answer: major heart attacks


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

CS:  0.5405248999595642
Answer:  d = 1.5.Explanation: To solve this problem, we need to use the distributive property of multiplication over subtraction. This means that we can multiply each term inside the parentheses by d and then subtract the results. So we have:
Target:  Answer: 84


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.3542213439941406
Answer:  Platypus
Explanation: The word 'eft' is used to refer to an animal that lays eggs and has a bill like a duck, webbed feet like an otter, a body of a beaver and the ability to produce milk. This animal is also known as a 'duck-beaver-otter'. It is found only in Australia and New Guinea. The animal was first described in 1799 by an English naturalist named George Shaw. He named the animal 'Ornithorhynchus anatinus' which means 'bird-snouted duck'. The name was later changed to 'Echidna' by a German zoologist named Johann Jakob Kaup in 1858. It was then renamed 'Platypodion' and finally to its current name, 'Pseudorhabdosynochus'.
Target:  Answer: Eft


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.8469570875167847
Answer:  A molecule is an electrically neutral group of two or more atoms held together by covalent chemical bonds. The word "molecule" comes from a Latin word meaning "little mass" or "small body".
Explanation: The question is asking for the definition of molecule, which is defined as "a small particle of a chemical element or compound". This definition does not specify the number of atoms, so the answer is "two atoms". The other choices are incorrect because they do not match this definition.
Target:  Answer: diatomic
CS:  0.6176376342773438
Answer:  Good hygiene
Explanation: The statement says "Eating a lot of food at one time is not good for your health." This means that if you eat a little bit at a time, your body will be able to digest the food better and you will stay healthy. This is because eating too much food all at once can make you feel sick and can also cause problems with your digestion. Therefore, eating many small meals throughout the day can he

In [ ]:
print(cosine/50)

0.5627617236599326


Few-shots CoT

In [ ]:
def few_shots_CoT(text):
    return f"""Answer each question. Then give a short explanation (1-2 sentences).

Q: {data[0]['input']}
Answer: from plants
Explanation: {data[0]['explanation']}

Q: {data[1]['input']}
Answer: Because metals conduct heat away from the skin faster
Explanation: {data[1]['explanation']}

Q: {data[2]['input']}
Answer: {data[2]['output']}
Explanation: {data[2]['explanation']}


Q: {text}
Answer: (clearly)"""


In [ ]:
clear_gpu()

In [ ]:
acc=0
cosine = 0
for i in range(2, 52):
  prompt = few_shots_CoT(data[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  input_len = input['input_ids'].shape[1]
  output = generate_output(type=None, input = input)
  generate_ids = output[0][input_len:]
  answer = tokenizer.decode(generate_ids, skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, data[i]['explanation'])
  cosine = cosine + c
  print(c)
  print('Answer: ', answer)
  print('Explanation (target): ', data[i]['explanation'])
  if extract_answer(answer, extract_target(data[i]['output'])):
    acc = acc+1
  print('Target: ', data[i]['output'])
  print('------------------------------------------------------------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.7927586436271667
Answer:  "Lee surrendered to Grant at Appomattox Court House, Virginia, on April 9, 1865, effectively ending the American Civil War."
Explanation (target):  The Battle of Appomattox Court House was a battle in the final stages of the American Civil War, resulting in Confederate General Robert E. Lee surrendering his Army to Union Commander Ulysses S. Grant on April 9th 1865
Target:  Answer: Battle of Appomattox Court House
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.7672989368438721
Answer:  Nasser spent 6 weeks in bed, while he suffered 2 heart-attacks. So, 6 is greater than 2. Therefore, 2 is less than 6. Answer: 2
Explanation (target):  The answer is the number of major heart attacks.
Though he was on bed rest for six weeks after his second attack, that's not greater than the total number of times (i.e., two) Nasser suffered from a heart attack.
Target:  Answer: major heart attacks
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

0.5727314949035645
Answer:  7644=4648d-4557d
Subtracting 4657d from both sides, we get
1497d=1957
Dividing by 1497,
d = 1.5
Explanation (target):  We need to isolate the variable d on one side of the equation.

[STEP 1] 7644 + 4557*d = 4648*d
[STEP 2] 12201 = 9205*d
[STEP 3] 12201/9205 = d
[STEP4] d= 84
Target:  Answer: 84
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.5365639925003052
Answer:  "The answer to your question is that efts are the larval stage of frogs and toads. They are also known as tadpoles."
Explanation (target):  An eft is an alternative name for Eft.
Target:  Answer: Eft
------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.8195717334747314
Answer:  "A molecule is defined as a group of two or more atoms held together by covalent chemical bonds. A molecule can consist of only one type of atom (monatomic) or of different types of atoms (diatomic, triatomic, etc.)."
Explanation (target):  A molecule is an electrically neutral group of two or more atoms held together by chemical bonds. If a molecule has only 2 atoms, then it is called diatomic. The answer to this question is therefore diatomic.
Target:  Answer: diatomic
------------------------------------------------------------
0.5051108002662659
Answer:  Eating a lot of food at one time is not good for your health. It can cause indigestion and make you feel sick. That's why it's better to eat smaller portions throughout the day. This way, your body can digest the food more easily and you won't feel bloated or uncomfortable. Plus, eating small, frequent meals can help you maintain a healthy weight and keep your energy levels up. So, if you want to stay he

In [ ]:
#ACC:
print(cosine/50)

0.5327847971394658


Few-shots CoT SC

In [ ]:
def few_shots_CoT_SC(text, output, num_samples=5, max_len=100, do_print=False):
    prompt = few_shots_CoT(text)
    samples = self_consistency(prompt = prompt,num_samples= num_samples, max_len = max_len)

    total_cos = 0
    scores = []

    for sample in samples:
        score = cosine_similarity_score(sample, output)
        total_cos += score
        scores.append(score)
        if do_print:
            print('Sample:', sample)
            print('Cosine:', score)
            print()

    avg_score = total_cos / len(samples) if samples else 0
    return samples, avg_score


In [9]:
from collections import Counter

def self_consistency(prompt, num_samples=5, max_len=100):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        input_len = inputs["input_ids"].shape[1]

        output = model.generate(
            **inputs,
            max_new_tokens=max_len,
            do_sample=True,
            temperature=0.8,
            top_k=50,
            no_repeat_ngram_size=3,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id  # tránh warning cho decoder-only
        )

        generated_ids = output[0][input_len:]
        de = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        outputs.append(de)

    return outputs


In [ ]:
cosine=0
for i in range(2, 52):
  _,c = few_shots_CoT_SC(data[i]['input'], data[i]['explanation'], num_samples=5, max_len = 250, do_print=True)
  if c != None:
    cosine = cosine + c
  print('Target: ', data[i]['output'])
  print('Explanation (target): ', data[i]['explanation'])
  print('--------------------------------------------------------------------')

Sample: Answer: Appomathtte Court House, Virginia
Exercise: Complete these sentences with the correct word.
1) The most important _ of the Civil War was that it ended slavery in the United States. (decision)
2) The Civil War began in 1861 when the southern states of _ seceded from the Union. (North)
3) Abraham Lincoln was the 16th _ who served as the President of the United Sates. (President)
4) During the Civil Rights Movement of the 1960s, the _ Congress passed the Civil rights Act of 1964 to protect the rights of African Americans. (Congress)
5) Rosa Parks was a _ woman who refused to give up her seat on a bus in Montgomery, Alabama in 1955. (civil rights)
6) The Emancipation Proclamation was a __ issued by President Abraham Lincoln in 1863 that declared all slaves in Confederate-held states to be free. (presidential)
7) The Gettysburg Address is a famous __ given by President John F. Kennedy in 1963 to commemorate the 100th anniversary of the end of the Union's Civil War. (speech)


KeyboardInterrupt: 

acc =
ap_acc =

In [ ]:
print(cosine/250)

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
task_samples = data[:100]
new_tasks = []
for task in task_samples:
    task["chain_of_thought"] = task['explanation']
    task["answer"] = task['output']
    new_tasks.append(task)
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)
def find_top_k_tasks(text, k=3):
        q = vectorizer.transform([text])
        scores = cosine_similarity(q, X).flatten()
        top = scores.argsort()[::-1][:k]
        return [new_tasks[i] for i in top]
def find_similar_task(query):
    q = vectorizer.transform([query])
    idx = cosine_similarity(q, X).argmax()
    return new_tasks[idx]
test_tasks = data[100:]

In [ ]:
  def build_prompt(examples, query):
        prompt = ""
        for ex in examples:
            prompt += f"""
            Input: {ex['input']}
            Answer: {ex['output']}
            """
            prompt += (
                "Now solve this task:\n"
                f"Input: {query}\n"
                "Answer: Let's solve this step-by-step. Write your reasoning clearly in 1-2 sentences."
            )
        return prompt

Few-shots CoT ART

In [ ]:
def few_shots_CoT_ART(text, k=1):
        examples = find_top_k_tasks(text, k)
        return build_prompt(examples, text)
acc=0
for i in range(50):
  prompt = few_shots_CoT_ART(test_tasks[i]['input'])
  input = tokenizer(prompt, return_tensors='pt').to('cuda')
  input_len = input['input_ids'].shape[1]
  output = generate_output(type=None, input = input)
  generate_ids = output[0][input_len:]
  answer = tokenizer.decode(generate_ids, skip_special_tokens=True).strip()
  c = cosine_similarity_score(answer, test_tasks[i]['explanation'])
  cosine = cosine + c
  print(c)
  print('Answer: ', answer)
  print('Explanation (target): ', test_tasks[i]['explanation'])
  print('Target: ', test_tasks[i]['output'])
  print('-----------------------------------------------------------')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.5662333965301514
Answer:  First, we need to understand what the task is asking for. It's asking us to create a title that summarizes the content of an article. So, let's read the article and see what it's about. From the excerpt, it seems like the focus is on a heated exchange between CNN's Cuomo, the President's adviser Conway and the recent shooting in Vegas. Therefore, a good title could be something like "Heated Exchange Between CNN Anchor and Trump Adviser on Gun Control After Vegas Shooting".
Explanation (target):  The main idea of the text is that Kellyanne Conway blames Hillary Clinton for not bringing up gun control. Thus, a title that conveys this point would be "Watch Chris Cuomo nail Kellyanne Conway for trying to blame lack of gun control on Hillary Clinton".
Target:  Answer: Watch Chris Cuomo nail Kellyanne Conway for trying to blame lack of gun control on Hillary Clinton
-----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.5830960869789124
Answer:  ```python
# Solution to Exercise 1
def solve_exercise_1():
    # We can infer that medesanos location is not near Canada because the paragraph does not mention anything about Canada or its location in relation to the town. The only information we have is that it's located in Italy, which is far from Canada. Therefore, we can't say for sure whether or not the two places are near each other, but we do know that they are not. 

    return "No"
Explanation (target):  The context indicates that Medesano is located in Emilia-Romagna, which is in the northern part of Italy. Canada would be well outside this region and therefore cannot be considered near to it. So, the answer should be "No".
Target:  Answer: No
-----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

0.43779391050338745
Answer:  First, we need to understand the task. It's asking us to check if the given title matches with the abstract. If it does, then the answer is 'yes', otherwise it's 'no'.
Now, let's look at the provided abstract and title. We can see that the title talks about 'PrimPol' and its role in DNA repair. However, our abstract does not mention anything about this enzyme. So, based on this information, what do you think is the correct answer?
Explanation (target):  The title summarizes the main idea of the paper, which is that PrimPol can reinitiate DNA synthesis after dNTP depletion and facilitate fork progression in UV-irradiated cells. It's a concise summary of what was studied and observed by the researchers who wrote this article. So "yes" is an appropriate answer.
Target:  Answer: yes
-----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.32522982358932495
Answer:  First, we need to identify the main characters and their roles. The main character is a reporter who gets fired and is trying to find out who killed her boss. There are also three men who are involved in a bank heist and are planning to blame it on the reporter's sister. Finally, there's a police detective who has also been fired from his job and has now started his own detective agency.
Explanation (target):  The movie plot is the story of a Japanese film called The Hole (洞, Ana), directed by Hiroshi Teshigahara. The movie was released in 1960 and can be found on YouTube as well.
Target:  Answer: The Hole
-----------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=400) and `max_length`(=400) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


0.4971555471420288
Answer:  First, we need to understand what the task is asking for. It's asking us to categorize a comment based on its level of explicit sexual content. This is a subjective task, as what one person might consider explicit, another might not. However, there are some general guidelines we can use to help us make a decision. For example, comments that use explicit language or imagery, or that describe sexual acts in detail, are likely to be considered more explicit than those that are more general or vague. Additionally, it's important to remember that what is considered appropriate or acceptable in one context may not be in another, so we should take into account the overall tone and subject matter of each comment when making our determination. With these guidelines in mind, let's take a look at RR's comment and see if it meets the criteria for being categorized as "sexually explicit".
Explanation (target):  The given comment is about Amazon opening a retail store in 

In [ ]:
print(cosine/50)

0.6112295862212777


In [63]:
clear_gpu()

In [20]:
import re

def get_best_thought(thoughts, text, do_print=False):
    output = select_best_path(thoughts, text, do_print=do_print)
    if 0<len(output) < 3:
        match = re.search(r"\b(\d+)\b", output)
        if match:
          index = int(match.group(1)) - 1
          if 0 <= index < len(thoughts):
              return thoughts[index]
    else:
        return output

In [51]:
def expand_thoughts(prompt, n=3):
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    input_len = inputs["input_ids"].shape[1]

    outputs = [model.generate(
        **inputs,
        max_new_tokens=100,
        pad_token_id=tokenizer.eos_token_id
    ) for _ in range(n)]

    return [
        tokenizer.decode(output[0][input_len:], skip_special_tokens=True).strip()
        for output in outputs
    ]
def recursive_expand_tree(prompt, depth, breadth, context, do_print=False, node_counter=[0], max_nodes=30):
            if depth == 0 or node_counter[0] >= max_nodes:
                return [prompt]

            base_prompt = f"""Instruction: Use tree-of-thought reasoning to explore solutions.

                Question: {context}

                Partial Reasoning:
                {prompt}
                Now expand with next steps:"""

            expanded = expand_thoughts(base_prompt, n=breadth)
            expanded = list(set(expanded))

            if do_print:
                print(f"[Depth {depth}] Base prompt:\n{base_prompt}")
                print(f"[Depth {depth}] Got thoughts:\n", expanded)

            tree = []
            for thought in expanded:
                if node_counter[0] >= max_nodes:
                    break
                node_counter[0] += 1
                sub_tree = recursive_expand_tree(thought, depth - 1, breadth, context, do_print, node_counter, max_nodes)
                tree.extend(sub_tree)
            return tree

def select_best_path(thoughts, text, do_print=False):
    prompt = f"""Instruction: Given a question and several reasoning paths, choose the most logical one.

    Question: {text}

    Candidates:
    {chr(10).join([f"{i + 1}. {t}" for i, t in enumerate(thoughts)])}

    Reply with the best option, justify briefly. Final answer is ?"""
    if do_print:
            print('prompt:\n', prompt)
    print(thoughts)
    input = tokenizer(prompt, return_tensors='pt').to('cuda')
    input_len = input['input_ids'].shape[1]
    output = generate_output(type=None, input = input, max_len = 100)
    generate_ids = output[0][input_len:]
    answer = tokenizer.decode(generate_ids, skip_special_tokens=True).strip()
    return answer

def zero_shot_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = zero_shot_ToT(text)
    if do_print:
            print('Root_prompt:\n', root_prompt)
    tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, context=text, do_print=do_print, max_nodes=30)
    best = get_best_thought(tree, text, do_print)
    return best

def few_shots_ToT_expanded(text, depth=2, breadth=3, do_print=False):
    root_prompt = few_shots_ToT(text)
    if do_print:
        print('[Root prompt]')
        print(root_prompt)

    tree = []
    for _ in range(breadth):
        single_tree = recursive_expand_tree(root_prompt, depth=depth, breadth=breadth, context=text, do_print=do_print, max_nodes=30)
        tree.extend(single_tree)

    best_path = get_best_thought(tree, text, do_print)
    return best_path


In [47]:
def zero_shot_ToT(text):
    return f"""
Instruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.

Question: {text}
Thought 1:
Thought 2:
Thought 3:

Answer: Explanation:
"""


In [52]:
acc = 0
cosine = 0
for i in range(50):
  answer = zero_shot_ToT_expanded(data[i]['input'], depth=2, breadth=2, do_print=False)
  c = cosine_similarity_score(answer, data[i]['explanation'])
  print('CS: ', c)
  cosine = cosine + c
  print('Answer: ', answer)
  print('Target: ', data[i]['explanation'])
  print('Output: ', data[i]['output'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


['Question: Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and']
CS:  0.6653151512145996
Answer:  Output: The correct option is A. Plants. This is because the article states that some plants produce compounds that can be used as medicines or drugs for various purposes. The other options are either incorrect or irrelevant to the question.
Target:  The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants menti

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


['Thought 2:\n                Now expand with next steps:\n                \n                Thought 3:\n                Now expand with next steps:\n                \n                Thought 1:\n                Now expand with next steps:\n                \n                Thought 2:\n                Now expand with next steps:\n                \n                Thought 3:\n                Now expand with next steps:\n                \n                Thought 1:\n                Now expand with next steps:']
CS:  0.5297049880027771
Answer:  <|question_end|>Tutor: Alright, let's break this down. First, we need to understand what each category of legal act means. Can you tell me what a regulation, a decision, and a directive are?
Target:  This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.
Output: 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

CS:  0.5644882917404175
Answer:  Output: Thought 1 is the correct answer, because it is based on factual information about the history and construction of wells. Thought 2 and 3 are irrelevant or incorrect because they do not explain where the water comes from, but rather how the wells are built or used. A possible justification is:
Target:  To answer the question, consider the following: A water well is an excavation or structure created in the ground by digging, driving, boring, or drilling to access groundwater in underground aquifers.
Output:  Answer: groundwater in underground aquifers
["\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nQuestion: Given a story, answer the question about the story. The question is the last sentence in the input. The story has one of the three following scenarios: (1) when the individual's belief matches reality, (2) when the individual's belie

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.07268260419368744
Answer:  
Target:  Mia's belief about the location of the celery does not match reality, but William's belief (red_bottle) about Mia's belief matches reality.
Output:  Answer: red_bottle
['\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nQuestion: Solve this math problem\n\nSolve 7644 = 4648*d - 4557*d for d. Choices: \nThought 1:\nThought 2:\nThought 3:\n\nAnswer: Explanation: \n']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.6491731405258179
Answer:  Output: 1. First, we can factor out d from both terms on the right side of the equation. This gives us d(7644-4487) =
Target:  We need to isolate the variable d on one side of the equation.

[STEP 1] 7644 + 4557*d = 4648*d
[STEP 2] 12201 = 9205*d
[STEP 3] 12201/9205 = d
[STEP4] d= 84
Output:  Answer: 84
["\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nQuestion: I've always wondered: An eft is an alternative name for which creature? Choices: \nThought 1:\nThought 2:\nThought 3:\n\nAnswer: Explanation: \n"]
CS:  0.44528287649154663
Answer:  Output: Thought 1 is incorrect because efts are not creatures, they are the larval stage of amphibians. Thought 2 is also incorrect for the same reason. However, Thought 3 is correct. Efts can refer to the larvae of frogs, toads, newts, and salamanders. They are aquatic and breathe through gills. Therefore, the

In [53]:
print(cosine/50)

0.3278489915654063


In [56]:
def zero_shot_ToT(text):
    return f"""
Instruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.

Input: {text}
Thought 1:
Thought 2:
Thought 3:

Output: Explanation: Final answer:
"""


In [57]:
acc = 0
cosine = 0
for i in range(50):
  answer = zero_shot_ToT_expanded(data[i]['input'], depth=2, breadth=2, do_print=False)
  c = cosine_similarity_score(answer, data[i]['explanation'])
  print('CS: ', c)
  cosine = cosine + c
  print('Answer: ', answer)
  print('Target: ', data[i]['explanation'])
  print('Output: ', data[i]['output'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


['\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nInput: Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl ester of salicylic acid, originally isolated from the bark of willow trees, and a wide range of opiate painkillers like heroi

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.6296406984329224
Answer:  A: Thought 1 is incorrect because it is too broad and vague. It does not specify which plants produce which drugs or how they are extracted or modified. Thought 2 is partially correct, but it leaves out some important examples of drugs that are not derived from plant sources, or that have complex chemical structures that make them difficult to isolate or synthesize. Therefore, thought 1 and thought 2 are both incorrect. The correct option is thought 3, which provides a more detailed and accurate description of the sources
Target:  The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants.

Therefore, "from plants" is the correct answer option to this question based o

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

CS:  0.7518641948699951
Answer:  Answer: 1. Water for wells comes from underground water sources. This is the correct answer, as wells are designed to tap into groundwater, which is water that fills the spaces between rocks and soil underground. Groundwater is replenished by rain and snowmelt that seeps through the soil and rock layers and reaches the water table, where it can be accessed by wells. Well water can also be contaminated by pollutants or bacteria, so it is important to test and treat it before using it for
Target:  To answer the question, consider the following: A water well is an excavation or structure created in the ground by digging, driving, boring, or drilling to access groundwater in underground aquifers.
Output:  Answer: groundwater in underground aquifers
["\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nInput: Given a story, answer the question about the s

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.5297071933746338
Answer:  Answer: Thought 1 is correct. In this scenario, William believes that there is a red bottle and a bucket, but he doesn't know which one Mia is looking for. Therefore, he thinks that she will search for it in either the bottle or the bucket. This is consistent with his own belief and the information he has. Thought 2 is incorrect, because it implies that William knows for sure which object Mia will look for, which is not the case. He only knows that it is either
Target:  Mia's belief about the location of the celery does not match reality, but William's belief (red_bottle) about Mia's belief matches reality.
Output:  Answer: red_bottle
['\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nInput: Solve this math problem\n\nSolve 7644 = 4648*d - 4557*d for d. Choices: \nThought 1:\nThought 2:\nThought 3:\n\nOutput: Explanation: Final answer: \n']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


CS:  0.5023140907287598
Answer:  Answer: The correct option is (1) because it simplifies the equation and isolates d on one side. The other options are incorrect because they do not follow the order of operations or cancel out the terms correctly. For example, (2) and (3) both add or subtract the same terms on both sides, which does not change the value of d or the equality. (4) is the correct way to solve the problem, but it is not one of the given choices.
Target:  We need to isolate the variable d on one side of the equation.

[STEP 1] 7644 + 4557*d = 4648*d
[STEP 2] 12201 = 9205*d
[STEP 3] 12201/9205 = d
[STEP4] d= 84
Output:  Answer: 84
["\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nInput: I've always wondered: An eft is an alternative name for which creature? Choices: \nThought 1:\nThought 2:\nThought 3:\n\nOutput: Explanation: Final answer: \n"]
CS:  0.5338857173919678

Acc =

In [66]:
def few_shots_ToT(text):
    return f"""
Instruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.

Input: {data[0]['input']}
Thought 1 (Common sense reasoning):
We commonly know that many drugs like caffeine, nicotine, and morphine are derived from natural sources like tea, tobacco, and poppy plants. These are familiar examples that most people recognize as plant-derived substances.

Thought 2 (Assumptions and implications):
The question implies a general source for medicines and recreational drugs. The context gives multiple examples of plant-based compounds, and no examples from animals or synthetic-only processes. So it's implied that plants are a primary source. If the answer were "animals" or "synthetic only," the text would need to mention them, but it does not.

Thought 3 (Cause-effect analysis):
Plants undergo secondary metabolism which produces chemical compounds like alkaloids. These have strong physiological effects on humans (e.g., caffeine stimulates the nervous system). That biological capability explains why plants are a common origin of such drugs.

Final Answer: from plants

---

Input: {data[1]['input']}
Thought 1 (Common sense reasoning):
Touching a metal door handle feels much colder than touching a wooden table, even if both have been in the same room. This everyday experience suggests metal pulls heat away from us more quickly.

Thought 2 (Assumptions and implications):
The assumption is that both materials are at the same temperature, so the cause of the sensation must be related to heat transfer, not temperature. The implication is that perception of "coldness" comes from heat leaving our body faster when we touch metal.

Thought 3 (Cause-effect analysis):
Metals are thermal conductors — they rapidly transfer heat from our warm skin into the cooler material. Wood is an insulator, so the heat stays near the contact area and doesn't flow away as fast, making it feel warmer to touch.

Final Answer: Because metals conduct heat away from the skin faster

---
Input: {text}
Thought 1:
Thought 2:
Thought 3:
 Final answer:


"""


In [71]:
clear_gpu()

In [72]:
acc = 0
cosine = 0
for i in range(50):
  answer = few_shots_ToT_expanded(data[i]['input'], depth=2, breadth=2, do_print=False)
  c = cosine_similarity_score(answer, data[i]['explanation'])
  print('CS: ', c)
  cosine = cosine + c
  print('Answer: ', answer)
  print('Target: ', data[i]['explanation'])
  print('Output: ', data[i]['output'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


['\nInstruction: For each question, explore multiple lines of reasoning. You must explain clearly in each path. Only after that, you give the final answer.\n\nInput: Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl ester of salicylic acid, originally isolated from the bark of willow trees, and a wide range of opiate painkillers like heroi

OutOfMemoryError: CUDA out of memory. Tried to allocate 308.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 244.12 MiB is free. Process 19611 has 14.50 GiB memory in use. Of the allocated memory 13.35 GiB is allocated by PyTorch, and 1.02 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(cosine/50)

Acc =

In [ ]:
acc = 0
cosine = 0
for i in range(50):
  answer = few_shots_ToT_expanded(data[i]['input'], depth=2, breadth=2, do_print=False)
  if answer != None:
    if extract_answer(answer,data[i]['output']):
      acc = acc + 1
    c = cosine_similarity_score(answer, data[i]['explanation'])
    print('CS: ', c)
    cosine = cosine + c
  print('Answer: ', answer)
  print('Target: ', data[i]['explanation'])
  print('Output: ', data[i]['output'])

In [ ]:
print(cosine/50)